In [1]:
import time 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

In [5]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")  # Prevents detection as bot
options.add_argument("--start-maximized")  # Opens browser in full screen
#options.add_argument("--incognito")  # Opens browser in Incognito mode
options.add_argument("--disable-popup-blocking")  # Prevents blocking of popups
options.add_argument("--disable-infobars")  # Removes 'Chrome is being controlled by automated test software'
options.add_argument("--disable-gpu")  # Reduces detection based on GPU usage
options.add_argument("--no-sandbox")  # Bypass OS security model
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

In [7]:
s = Service(service = '/Users/sehajsukhleensingh/.wdm/drivers/chromedriver/mac64/133.0.6943.126/chromedriver-mac-arm64/chromedriver')

In [9]:
data = []

In [27]:
driver = webdriver.Chrome(service = s,options = options )
num = 92
start = num
end = num + 13
while start <= end:
    driver.get(f'https://www.99acres.com/search/property/buy?city=8&res_com=R&preference=S&property_type=2&page={start}')
    time.sleep(2)
    
    oldH = driver.execute_script('return document.body.scrollHeight')
    
    while True:
    
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(1)
    
        newH = driver.execute_script('return document.body.scrollHeight')
    
        if newH  == oldH:
            html = driver.page_source
            break
        oldH = newH
    
    soup = bs(html,'lxml')
    container = soup.find_all('div' , class_ = 'tupleNew__innerCont')
    
    for i in container:
        
        try:
            link = i.find('a',class_ ='tupleNew__propertyHeading ellipsis').get('href')
    
            try:
                name = i.find('h2',class_= 'tupleNew__propType').get_text()
            except:
                pass
            try:
                price = i.find('div',class_ = 'tupleNew__priceValWrap').find('span').text
            except:
                pass
                
        except:
            print('couldnt find the link')
            
        finally:
            dt = {'name':name , 'price':price , 'link':link}
            data.append(dt)
            
    
    print(f'scraped page : {start}')
    print(f'data items on page: {start} -> {len(container)}')
    
    start += 1
    
driver.quit()

scraped page : 92
data items on page: 92 -> 26
scraped page : 93
data items on page: 93 -> 26
scraped page : 94
data items on page: 94 -> 26
scraped page : 95
data items on page: 95 -> 26
scraped page : 96
data items on page: 96 -> 26
scraped page : 97
data items on page: 97 -> 26
scraped page : 98
data items on page: 98 -> 26
scraped page : 99
data items on page: 99 -> 26
scraped page : 100
data items on page: 100 -> 26
scraped page : 101
data items on page: 101 -> 26
scraped page : 102
data items on page: 102 -> 26
scraped page : 103
data items on page: 103 -> 26
scraped page : 104
data items on page: 104 -> 24
scraped page : 105
data items on page: 105 -> 0


In [29]:
len(data)

3582

In [31]:
df = pd.DataFrame(data)

In [33]:
df

,name,price,link
0,"3 Bedroom House in Sector 4, Gurgaon",₹3.4 Cr,https://www.99acres.com/3-bhk-bedroom-independ...
1,"3 Bedroom House in Sector 4, Gurgaon",₹3.2 Cr,https://www.99acres.com/3-bhk-bedroom-independ...
2,"3 Bedroom House in New Colony, Gurgaon",₹3.25 Cr,https://www.99acres.com/3-bhk-bedroom-independ...
3,"4 Bedroom House in Sector 9, Gurgaon",₹6 Cr,https://www.99acres.com/4-bhk-bedroom-independ...
4,"7 Bedroom House in DLF Phase 2, Gurgaon",₹22.5 Cr,https://www.99acres.com/7-bhk-bedroom-independ...
...,...,...,...
3577,"12 Bedroom House in Rajiv Chowk, Gurgaon",₹2.05 Cr,https://www.99acres.com/12-bhk-bedroom-indepen...
3578,"5 Bedroom House in DLF Phase 1, Gurgaon",₹12 Cr,https://www.99acres.com/5-bhk-bedroom-independ...
3579,"4 Bedroom House in Sector 109, Gurgaon",₹12 Cr,https://www.99acres.com/4-bhk-bedroom-independ...
3580,"7 Bedroom House in DLF Phase 1, Gurgaon",₹37 Cr,https://www.99acres.com/7-bhk-bedroom-independ...


In [37]:
df = df[~df.duplicated()]

In [39]:
df.duplicated().sum()

0

In [41]:
df.to_csv('links-gurgaon-homes.csv')